In [ ]:
from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader

from odc.stac import configure_s3_access

from dep_tools.grids import PACIFIC_GRID_10
from src.utils import GeoMADSentinel2Processor

from dask.distributed import Client as DaskClient

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up rasterio
configure_s3_access(aws_unsigned=True)

In [ ]:
# Study site configuration
# item_id = "63,20"  # West viti levu
# item_id = "65,22"  # Near the anti-meridian
# item_id = "66,22"  # Right of the anti-meridian
# item_id = "48,16"  # Vanuatu, middle of Grand Terre
# item_id = "89,16"  # Rarotonga
# item_id = "57,43"  #  Kiribati, Tarawa
# item_id = "56,49"  # atoll with thin land which S-2 thinks is cloud/cloud-shadow

# item_id = "66,20"  # Area across am that is failing. TODO: find out why!
item_id = "65,20"  # To the west/left of the above tile

# No items found
item_id = "50,15"
datetime = "2018"

# datetime = "2024"

# And get the study site
tile_index = tuple(int(i) for i in item_id.split(","))
geobox = PACIFIC_GRID_10.tile_geobox(tile_index)

# Load low-res if you want it faster
geobox = geobox.zoom_out(10)

geobox.explore()

In [ ]:
# Search for data
searcher = PystacSearcher(
    catalog="https://earth-search.aws.element84.com/v1",
    collections=["sentinel-2-l2a"],
    datetime=datetime,
)

items = searcher.search(area=geobox)
print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    # bands=["SCL", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    bands=["scl", "red", "blue", "green"],
    chunks=dict(time=1, x=3201, y=3201),
    groupby="solar_day",
    fail_on_error=False,
    nodata=0
)

# Run the load process, which is lazy-loaded
input_data = loader.load(items, areas=geobox)
input_data

In [ ]:
items[n]

In [ ]:
from odc.stac import load

n = 18
data = load([items[n]], geobox=geobox, measurements=["red", "green", "blue"], chunks={})

data.isel(time=0).to_array().plot.imshow(robust=True)

# data.isel(time=0).to_array().odc.write_cog("rgb_test.tif")

In [ ]:
items[5]

In [ ]:
median = input_data[["red"]].median(dim="time")

median.red.plot.imshow(robust=True, figsize=(10, 10))

In [ ]:
# Set up a data processor
processor = GeoMADSentinel2Processor(
    geomad_options=dict(
        work_chunks=(600, 600),
        num_threads=10,
        maxiters=100,
        scale=1/10_000,
    ),
    min_timesteps=5,
    drop_vars = ["scl"],
)

# Do the processing using a local dask cluster
with DaskClient(n_workers=1, threads_per_worker=32, memory_limit="10GB"):
    output_data = processor.process(input_data)

output_data

In [ ]:
output_data.odc.explore(robust=True)

In [ ]:
output_data[["red", "green", "blue"]].to_array().plot.imshow(vmin=0, vmax=2000, size=10)

In [ ]:
import json

with open("geobox.geographic_extent.json", "w") as f:
    f.write(json.dumps(geobox.geographic_extent.json, indent=2))